
# TIANSHOU

In [3]:
import datetime
import os
import pprint

import numpy as np
import torch
import glob

from torch.utils.tensorboard import SummaryWriter
from tianshou.data import Collector, CollectStats, ReplayBuffer, VectorReplayBuffer
from tianshou.exploration import GaussianNoise
from tianshou.highlevel.logger import LoggerFactoryDefault
from tianshou.policy import TD3Policy
from tianshou.policy.base import BasePolicy
from tianshou.trainer import OffpolicyTrainer
from tianshou.utils.net.common import Net
from tianshou.utils.net.continuous import Actor, Critic
from tianshou.env import DummyVectorEnv
from tianshou.utils import TensorboardLogger

import gymnasium as gym

from gymnasium.wrappers import RecordVideo


In [4]:
env = gym.make('LunarLanderContinuous-v2', render_mode="human")

train_envs = DummyVectorEnv([lambda: gym.make('LunarLanderContinuous-v2') for _ in range(10)])
test_envs = DummyVectorEnv([lambda: gym.make('LunarLanderContinuous-v2') for _ in range(10)])

state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
max_action = env.action_space.high[0]

print("Observations shape:", state_shape)
print("Actions shape:", action_shape)
print("Action range:", np.min(env.action_space.low), np.max(env.action_space.high))

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

Observations shape: (8,)
Actions shape: (2,)
Action range: -1.0 1.0


In [8]:
#Model
hidden_sizes = ()
device = "cuda" if torch.cuda.is_available() else "cpu"
net_a = Net(state_shape=state_shape, hidden_sizes=hidden_sizes, device=device)
actor = Actor(net_a, action_shape, max_action=max_action, device=device).to(
    device,
)
actor_optim = torch.optim.Adam(actor.parameters(), lr=1e-3)
net_c1 = Net(
    state_shape=state_shape,
    action_shape=action_shape,
    hidden_sizes=hidden_sizes,
    concat=True,
    device=device,
)
net_c2 = Net(
    state_shape=state_shape,
    action_shape=action_shape,
    hidden_sizes=hidden_sizes,
    concat=True,
    device=device,
)
critic1 = Critic(net_c1, device=device).to(device)
critic1_optim = torch.optim.Adam(critic1.parameters(), lr=1e-3)
critic2 = Critic(net_c2, device=device).to(device)
critic2_optim = torch.optim.Adam(critic2.parameters(), lr=1e-3)

policy: TD3Policy = TD3Policy(
    actor=actor,
    actor_optim=actor_optim,
    critic=critic1,
    critic_optim=critic1_optim,
    critic2=critic2,
    critic2_optim=critic2_optim,
    tau=0.005,
    gamma=0.99,
    exploration_noise=GaussianNoise(sigma=0.1),
    policy_noise=0.2,
    update_actor_freq=2,
    noise_clip=0.5,
    estimation_step=1,
    action_space=env.action_space,
    action_scaling=True
)

# Collector
buffer = VectorReplayBuffer(20000, 10)
train_collector = Collector(policy, train_envs, buffer, exploration_noise=True)
test_collector = Collector(policy, test_envs)
train_collector.reset()
train_collector.collect(n_step=1000, random=True)

# logger
writer = SummaryWriter(log_dir='log/td3_lunar_lander')
logger = TensorboardLogger(writer)

def save_best_fn(policy: BasePolicy) -> None:
    torch.save(policy.state_dict(), 'best_policy.pth')

result = OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=100,
    step_per_epoch=10000,
    step_per_collect=1000,
    episode_per_test=1000,
    batch_size=64,
    save_best_fn=save_best_fn,
    logger=logger,
    update_per_step=0.1,
    test_in_train=False,
    ).run()
pprint.pprint(result)


Epoch #1: 10001it [00:08, 1232.29it/s, env_step=10000, gradient_step=1000, len=80, n/ep=12, n/st=1000, rew=-415.69]                           


Epoch #1: test_reward: -619.778763 ± 145.154765, best_reward: -591.427869 ± 298.438357 in #0


Epoch #2: 10001it [00:08, 1218.10it/s, env_step=20000, gradient_step=2000, len=68, n/ep=17, n/st=1000, rew=-265.62]                           


Epoch #2: test_reward: -143.263332 ± 35.298072, best_reward: -143.263332 ± 35.298072 in #2


Epoch #3: 10001it [00:07, 1254.25it/s, env_step=30000, gradient_step=3000, len=74, n/ep=17, n/st=1000, rew=-340.74]                           


...

Epoch #98: test_reward: -422.675252 ± 81.099067, best_reward: -65.217313 ± 36.256643 in #17


Epoch #99: 10001it [00:08, 1120.75it/s, env_step=990000, gradient_step=99000, len=65, n/ep=16, n/st=1000, rew=-497.18]                           


Epoch #99: test_reward: -522.332434 ± 121.082275, best_reward: -65.217313 ± 36.256643 in #17


Epoch #100: 10001it [00:09, 1025.99it/s, env_step=1000000, gradient_step=100000, len=63, n/ep=13, n/st=1000, rew=-548.16]                          


Epoch #100: test_reward: -557.403129 ± 131.678031, best_reward: -65.217313 ± 36.256643 in #17
InfoStats(gradient_step=100000,
          best_reward=-65.21731272085137,
          best_reward_std=36.256643477845046,
          train_step=1000000,
          train_episode=12401,
          test_step=8607013,
          test_episode=101000,
          timing=TimingStats(total_time=2518.535719871521,
                             train_time=861.0854756832123,
                             train_time_collect=196.2006607055664,
                             train_time_update=634.9471399784088,
                             test_time=1657.4502441883087,
                             update_speed=1161.3248954252406))


In [9]:
# Let's watch its performance!
test_envs.seed(seed)
test_collector.reset()
collector_stats = test_collector.collect(n_episode = 10, render = True)
print(collector_stats)

CollectStats(n_collected_episodes=10, n_collected_steps=653, collect_time=101.52879357337952, collect_speed=6.431672996567688, returns=array([ -447.82449881,  -406.19764223,  -530.63178111,  -474.35582903,
        -387.41465344,  -535.55734933,  -529.98647276,  -613.17937989,
        -549.11003256, -1086.11987812]), returns_stat=SequenceSummaryStats(mean=-556.037751726602, std=188.49712234988263, max=-387.41465343624105, min=-1086.1198781225087), lens=array([ 51,  57,  58,  58,  58,  59,  64,  72,  75, 101]), lens_stat=SequenceSummaryStats(mean=65.3, std=13.740815114104404, max=101.0, min=51.0))


In [10]:
# lunvh tensorboard
from bbrl_utils.notebook import setup_tensorboard

setup_tensorboard("./log/")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 28284), started 5:29:49 ago. (Use '!kill 28284' to kill it.)